In [144]:
import sys
import os
sys.path.insert(1, '/home/jovyan/graphormer_v2/')

import torch
import numpy as np
import torch_geometric.datasets
from ogb.graphproppred import PygGraphPropPredDataset
from ogb.lsc.pcqm4m_pyg import PygPCQM4MDataset
import pyximport
from torch_geometric.data import InMemoryDataset, download_url
import pandas as pd
from sklearn import preprocessing

pyximport.install(setup_args={'include_dirs': np.get_include()})
import os.path as osp
from torch_geometric.data import Data
import time

from torch_geometric.utils import add_self_loops, negative_sampling
from graphormer.data.wrapper import preprocess_item
from torch_geometric.data import Dataset

In [157]:
class geo_Omsk(Dataset):
    
    def __init__(self, root, transform=None, pre_transform=None, split = 'train'):
        super(geo_Omsk, self).__init__(root, transform, pre_transform, split)
        self.split = split
        if '_upd' not in root:
            self.data, self.slices = torch.load(self.processed_dir + '/' + f'{split}.pt')
        # self.data = torch.load(self.processed_dir + '/' + f'{split}.pt')
#         self.raw_dir = '/home/jovyan/'
        
    @property
    def raw_dir(self) -> str:
        return '/home/jovyan/tte_data/'
    
    
    @property
    def raw_file_names(self):
        return ['omsk_full_routes_final_weather_L_NaN_filtered_FIXED.csv']

    @property
    def processed_file_names(self):
        return ['train.pt', 'test.pt', 'val.pt']
    
    @property
    def processed_dir(self):
        return osp.join(self.root)

    # def download(self):
    #     path = download_url(self.url, self.raw_dir)
    #     print(self.processed_paths[0])
    
    
#     def process(self):
#         # Read data
#         start_time = time.time()
#         data = pd.read_csv(osp.join('/home/jovyan/tte_data/', 'omsk_full_routes_final_weather_L_NaN_filtered_FIXED.csv'))
#         data = data[data['rebuildCount']<=1].reset_index(drop = True).copy()
#         shape = int(data.shape[0])
#         data = data[0:shape].copy()

#         data = data.drop(columns = ['Unnamed: 0'])
#         data['hour'] = data['start_timestamp'].apply(lambda x: int(x[-10:-8]))
#         # Graph 
#         graph_columns_gran = ['edges', 'time', 'speed', 'length']
#         edges = ['edges']
#         target = ['time']
#         node_features_gran = ['speed', 'length']

#         edge_features_agg = [' start_point_part', 'finish_point_part', 'day_period', 'week_period', 'clouds', 'snow', 'temperature', 'wind_dir', 'wind_speed', 'pressure','hour']


#         all_speed = []
#         all_length = []
#         for i in range(0,shape):
#             data_row = data[i:i+1].reset_index(drop = True).copy()
#             speed_list = [int(x) for x in (data_row['speed'].values[0].replace("'",'').split(','))]
#             list_length = [int(x) for x in (data_row['length'].values[0].replace("'",'').split(','))]
#             all_speed.append(speed_list)
#             all_length.append(list_length)

#         all_speed = [item for sublist in all_speed for item in sublist]
#         all_length = [item for sublist in all_length for item in sublist]

#         train_size = 0.8
#         test_size = 0.1
#         val_size = 0.1

#         data_split_dict = dict()
#         data_split_dict['train'] = np.arange(0, int(data.shape[0]*train_size))
#         data_split_dict['test'] = np.arange(int(data.shape[0]*train_size), int(data.shape[0]*(train_size+test_size)))
#         data_split_dict['val'] = np.arange(int(data.shape[0]*(train_size + test_size)),int((data.shape[0]*(train_size+test_size + val_size))))

#         for split in data_split_dict.keys():
#             data_list = []
#             for i in data_split_dict[split]:
#                 data_row = data.iloc[[i],].reset_index(drop = True).copy()

#                 edge_list = [int(x) for x in (data_row['edges'].values[0].replace("'",'').split(','))]
#                 speed_list = [int(x) for x in (data_row['speed'].values[0].replace("'",'').split(','))]
#                 list_length = [int(x) for x in (data_row['length'].values[0].replace("'",'').split(','))]

#                 source = edge_list.copy()
#                 target = edge_list[1:].copy() + [edge_list[0]].copy()

#                 data_row_gran = pd.DataFrame()
#                 data_row_gran['source'] = source
#                 data_row_gran['target'] = target
#                 data_row_gran['speed'] = speed_list
#                 data_row_gran['length'] = list_length


#                 target_val = data_row['RTA'].values[0]


#                 data_row_gran['speed'] = data_row_gran['speed']/np.mean(speed_list)
#                 data_row_gran['length'] = data_row_gran['length']/np.mean(list_length)

#                 for col in edge_features_agg:
#                     data_row_gran[col] = data_row[col].values[0]

#                 total_nodes_list = list(set(list(data_row_gran.source.values)))
#                 le = preprocessing.LabelEncoder()
#                 le.fit(total_nodes_list)

#                 data_row_gran['source'] = le.transform(data_row_gran.source.values)
#                 data_row_gran['target'] = le.transform(data_row_gran.target.values)

#                 total_nodes_list = list(set(list(data_row_gran.source.values)))

#                 edge_index = torch.tensor(torch.from_numpy(data_row_gran[['source','target']].values.T),dtype = torch.long)


#                 # Define tensor of nodes features
#                 x = torch.tensor(torch.from_numpy(data_row_gran[['speed','length'] + edge_features_agg].values),dtype = torch.long)

#                 # Define tensor of edge features

#                 # Define tensor of edge features
#                 edge_num_feach = 1
#                 edge_attr = torch.from_numpy(np.ones(shape = ((edge_index.size()[1]), edge_num_feach)))
#                 edge_attr = torch.tensor(edge_attr,dtype = torch.long)

#                 # Define tensor of targets
#                 y = torch.tensor(target_val,dtype = torch.long)


#                 data_graph = Data(x=x, edge_index = edge_index, edge_attr = edge_attr, y=y)
#                 data_list.append(data_graph)
#             torch.save(data_list, osp.join(self.processed_dir, f'{split}.pt'))

    
    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'{self.split}.pt'))
        return data[idx]


In [161]:
update_dir = osp.join(root, 'processed', 'data_omsk_1_upd')
geo_Omsk(update_dir,split = 'train').get(1)

Data(attn_bias=[73, 73], attn_edge_type=[72, 72, 1], edge_attr=[72, 1], edge_index=[2, 72], edge_input=[72, 72, 71, 1], in_degree=[72], out_degree=[72], spatial_pos=[72, 72], x=[72, 13], y=[1])

In [147]:
geo_Omsk.get('train')

TypeError: get() missing 1 required positional argument: 'idx'

In [105]:
def update_data_split(raw_data, update_dir, split):
    graph_count = len(raw_data)
    upd_list = list()
    for i in range(0, 10):
        if i%100 == 0:
            print(i)
        graph = preprocess_item(raw_data[i])
        upd_list.append(graph)
    if not os.path.exists(update_dir):
        os.makedirs(update_dir)
    torch.save(upd_list, osp.join(update_dir, f'{split}.pt'))
    
def update_data(raw_dir, update_dir):
    for split in ['train', 'test', 'val']:
        raw_data = geo_Omsk(raw_dir, split = split)
        update_data_split(raw_data, update_dir, split)
        
root = osp.join('dataset', 'omsk')
raw_dir = osp.join(root, 'processed', 'data_omsk_1')
update_dir = osp.join(root, 'processed', 'data_omsk_1_upd')
update_data(raw_dir, update_dir)

In [106]:
def update_data(raw_dir, update_dir):
    for split in ['train', 'test', 'val']:
        raw_data = geo_Omsk(raw_dir, split = split)
        update_data_split(raw_data, update_dir, split)

In [108]:
root = osp.join('dataset', 'omsk')
raw_dir = osp.join(root, 'processed', 'data_omsk_1')
update_dir = osp.join(root, 'processed', 'data_omsk_1_upd')
update_data(raw_dir, update_dir)

0
0
0


In [89]:
root = osp.join('dataset', 'omsk')
raw_dir = osp.join(root, 'processed', 'data_omsk_1')
update_dir = osp.join(root, 'processed', 'data_omsk_1_upd')
update_data(raw_dir, update_dir)

In [90]:
a.load_raw_data(split = 'train')

Data(edge_attr=[24401141, 1], edge_index=[2, 24401141], x=[24401141, 13], y=[431406])

In [96]:
osp.join(osp.join('dataset', 'omsk'), 'lolo')

'dataset/omsk/lolo'

In [70]:
class geo_Omsk_2(InMemoryDataset):
   
    @property
    def processed_dir(self, update = False):
        return osp.join(self.root, 'processed', 'data_omsk_1')
        
    def __init__(self, root, transform=None, pre_transform=None, split = 'train', update = False):
        super(geo_Omsk_2, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_dir + '/' + f'{split}.pt')
        self.update = update
        print(self.update)
        


In [68]:
a = geo_Omsk_2(root, split = 'train', update = True)

loloo
True


In [66]:
a

geo_Omsk_2(431406)

In [36]:
root = "dataset/" + 'omsk'
data_train_raw = geo_Omsk(root, split = 'train')
# data_test_raw = geo_Omsk(root, split = 'test')
# data_valid_raw = geo_Omsk(root, split = 'val')

TypeError: __init__() takes from 1 to 5 positional arguments but 6 were given

In [15]:
data_train_raw[0]

Data(edge_attr=[53, 1], edge_index=[2, 53], x=[53, 13], y=[1])

In [14]:
data_test_raw[0]

Data(edge_attr=[48, 1], edge_index=[2, 48], x=[48, 13], y=[1])

In [16]:
data_valid_raw[0]

Data(edge_attr=[31, 1], edge_index=[2, 31], x=[31, 13], y=[1])

In [13]:
data_train_raw[0]

Data(edge_attr=[53, 1], edge_index=[2, 53], x=[53, 13], y=[1])